<a href="https://colab.research.google.com/github/syed-irtiza7/SyedIrtiza_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [9]:
# 1) Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [4]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [10]:
# Q1 — Write your answer here

def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML."""
    # Read all tables from HTML
    tables = pd.read_html(html)

    # Find the first table with at least 3 columns
    for table in tables:
        if len(table.columns) >= 3:
            # Flatten multi-level headers if they exist
            df = flatten_headers(table)
            return df

    raise ValueError("No table with 3 or more columns found")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids."""
    # Make a copy to avoid modifying the original
    df_clean = df.copy()

    # Clean column names - strip whitespace
    df_clean.columns = [col.strip() for col in df_clean.columns]

    # Identify the correct columns (case-insensitive matching)
    column_mapping = {}
    for col in df_clean.columns:
        col_lower = col.lower()
        if 'country' in col_lower:
            column_mapping['Country'] = col
        elif 'alpha-2' in col_lower or 'alpha2' in col_lower.replace('-', ''):
            column_mapping['Alpha-2'] = col
        elif 'alpha-3' in col_lower or 'alpha3' in col_lower.replace('-', ''):
            column_mapping['Alpha-3'] = col
        elif 'numeric' in col_lower or 'code' in col_lower:
            column_mapping['Numeric'] = col

    # Select only the columns we need
    required_cols = ['Country', 'Alpha-2', 'Alpha-3', 'Numeric']
    available_cols = [column_mapping.get(col) for col in required_cols if column_mapping.get(col)]

    if len(available_cols) < 4:
        raise ValueError(f"Could not find all required columns. Found: {available_cols}")

    df_clean = df_clean[available_cols].copy()
    df_clean.columns = required_cols

    # Clean data: strip whitespace from string columns
    for col in ['Country', 'Alpha-2', 'Alpha-3']:
        df_clean[col] = df_clean[col].astype(str).str.strip()

    # Convert Alpha-2 and Alpha-3 to uppercase
    df_clean['Alpha-2'] = df_clean['Alpha-2'].str.upper()
    df_clean['Alpha-3'] = df_clean['Alpha-3'].str.upper()

    # Convert Numeric to integer (nullable)
    df_clean['Numeric'] = pd.to_numeric(df_clean['Numeric'], errors='coerce').astype('Int64')

    # Drop rows with missing critical values
    df_clean = df_clean.dropna(subset=['Country', 'Alpha-2', 'Alpha-3', 'Numeric'])

    # Remove any rows where country name is obviously invalid
    df_clean = df_clean[~df_clean['Country'].isin(['', 'Country', 'Country Name'])]

    return df_clean

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N."""
    return df.sort_values('Numeric', ascending=False).head(top)

# Execute Q1 pipeline
try:
    print("Starting Q1 - IBAN Country Codes scraping...")

    # Fetch HTML content
    url = "https://www.iban.com/country-codes"
    html_content = fetch_html(url)
    print("✓ HTML content fetched successfully")

    # Read and extract table
    raw_df = q1_read_table(html_content)
    print(f"✓ Raw table extracted with {len(raw_df)} rows and {len(raw_df.columns)} columns")
    print(f"  Columns found: {list(raw_df.columns)}")

    # Clean the data
    cleaned_df = q1_clean(raw_df)
    print(f"✓ Data cleaned: {len(cleaned_df)} valid rows remaining")

    # Get top 15 sorted by Numeric (descending)
    top_15 = q1_sort_top(cleaned_df, 15)

    # Save to CSV
    cleaned_df.to_csv('data_q1.csv', index=False)
    print("✓ Data saved to 'data_q1.csv'")

    # Display results
    print("\n" + "="*70)
    print("Q1 RESULTS - Top 15 Countries by Numeric Code (Descending)")
    print("="*70)
    print(top_15.to_string(index=False))
    print(f"\nTotal countries in dataset: {len(cleaned_df)}")

except Exception as e:
    print(f"❌ Error in Q1: {e}")
    import traceback
    traceback.print_exc()




Starting Q1 - IBAN Country Codes scraping...
✓ HTML content fetched successfully
✓ Raw table extracted with 249 rows and 4 columns
  Columns found: ['Country', 'Alpha-2 code', 'Alpha-3 code', 'Numeric']
✓ Data cleaned: 249 valid rows remaining
✓ Data saved to 'data_q1.csv'

Q1 RESULTS - Top 15 Countries by Numeric Code (Descending)
                                                   Country Alpha-2 Alpha-3  Numeric
                                                    Zambia      ZM     ZMB      894
                                                     Yemen      YE     YEM      887
                                                     Samoa      WS     WSM      882
                                         Wallis and Futuna      WF     WLF      876
                        Venezuela (Bolivarian Republic of)      VE     VEN      862
                                                Uzbekistan      UZ     UZB      860
                                                   Uruguay      UY     URY    

/tmp/ipython-input-1330224901.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [6]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [11]:
# Q2 — Write your answer here

def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    """
    soup = BeautifulSoup(html, 'html.parser')

    items = []

    # Find all story rows with class 'athing'
    story_rows = soup.find_all('tr', class_='athing')

    for story in story_rows:
        item_data = {}

        # Extract rank
        rank_span = story.find('span', class_='rank')
        item_data['rank'] = rank_span.text.strip().replace('.', '') if rank_span else ''

        # Extract title and link
        title_link = story.find('a', class_='titlelink')
        item_data['title'] = title_link.text if title_link else ''
        item_data['link'] = title_link.get('href', '') if title_link else ''

        # Find the next sibling row with class 'subtext' for metadata
        subtext_row = story.find_next_sibling('tr')
        points = '0'
        user = ''
        comments = '0'

        if subtext_row and subtext_row.find('td', class_='subtext'):
            subtext = subtext_row.find('td', class_='subtext')

            # Extract points
            score_span = subtext.find('span', class_='score')
            if score_span:
                points = score_span.text.replace(' points', '').replace(' point', '')

            # Extract user
            user_link = subtext.find('a', class_='hnuser')
            if user_link:
                user = user_link.text

            # Extract comments
            all_links = subtext.find_all('a')
            if len(all_links) >= 3:
                comments_link = all_links[-1]  # Last link is usually comments
                comments_text = comments_link.text
                if 'comment' in comments_text.lower():
                    # Extract number from text like "42 comments" or "discuss"
                    if 'discuss' in comments_text.lower():
                        comments = '0'
                    else:
                        comments = comments_text.split()[0]

        item_data['points'] = points
        item_data['user'] = user
        item_data['comments'] = comments

        items.append(item_data)

    return pd.DataFrame(items)

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values."""
    df_clean = df.copy()

    # Clean rank - remove non-numeric characters and convert to int
    df_clean['rank'] = df_clean['rank'].astype(str).str.replace(r'\D', '', regex=True)
    df_clean['rank'] = pd.to_numeric(df_clean['rank'], errors='coerce').fillna(0).astype(int)

    # Clean points - extract numbers only
    df_clean['points'] = df_clean['points'].astype(str).str.replace(r'\D', '', regex=True)
    df_clean['points'] = pd.to_numeric(df_clean['points'], errors='coerce').fillna(0).astype(int)

    # Clean comments - handle special cases and extract numbers
    df_clean['comments'] = df_clean['comments'].astype(str)

    # Replace "discuss" with 0 and extract numbers from comments text
    def clean_comment_text(text):
        text = str(text).lower().strip()
        if text in ['discuss', '']:
            return '0'
        # Extract first number found in the text
        numbers = re.findall(r'\d+', text)
        return numbers[0] if numbers else '0'

    df_clean['comments'] = df_clean['comments'].apply(clean_comment_text)
    df_clean['comments'] = pd.to_numeric(df_clean['comments'], errors='coerce').fillna(0).astype(int)

    # Fill missing text fields with empty strings
    text_columns = ['title', 'link', 'user']
    for col in text_columns:
        df_clean[col] = df_clean[col].fillna('').astype(str)

    # Strip whitespace from text fields
    for col in text_columns:
        df_clean[col] = df_clean[col].str.strip()

    return df_clean

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    return df.sort_values('points', ascending=False).head(top)

# Execute Q2 pipeline
try:
    print("Starting Q2 - Hacker News Front Page scraping...")

    # Fetch HTML content
    url = "https://news.ycombinator.com/"
    html_content = fetch_html(url)
    print("✓ HTML content fetched successfully")

    # Parse items from HTML
    raw_df = q2_parse_items(html_content)
    print(f"✓ Parsed {len(raw_df)} stories from Hacker News front page")

    # Clean the data
    cleaned_df = q2_clean(raw_df)
    print(f"✓ Data cleaned: {len(cleaned_df)} valid stories")

    # Get top 15 sorted by points (descending)
    top_15 = q2_sort_top(cleaned_df, 15)

    # Save to CSV
    cleaned_df.to_csv('data_q2.csv', index=False)
    print("✓ Data saved to 'data_q2.csv'")

    # Display results
    print("\n" + "="*90)
    print("Q2 RESULTS - Top 15 Hacker News Stories by Points (Descending)")
    print("="*90)

    # Format output for better readability
    display_df = top_15[['rank', 'title', 'points', 'comments', 'user']].copy()

    # Truncate long titles for better display
    display_df['title'] = display_df['title'].apply(
        lambda x: x[:70] + '...' if len(x) > 70 else x
    )

    print(display_df.to_string(index=False))
    print(f"\nTotal stories scraped: {len(cleaned_df)}")

except Exception as e:
    print(f"❌ Error in Q2: {e}")
    import traceback
    traceback.print_exc()



Starting Q2 - Hacker News Front Page scraping...
✓ HTML content fetched successfully
✓ Parsed 30 stories from Hacker News front page
✓ Data cleaned: 30 valid stories
✓ Data saved to 'data_q2.csv'

Q2 RESULTS - Top 15 Hacker News Stories by Points (Descending)
 rank title  points  comments         user
   13           502       184  WaitWaitWha
    9           311       211        art-w
   28           282        47    kirschner
   30           275       152    granzymes
    1           234       118    vitalnodo
    6           232        86  birdculture
    8           202        77 joemasilotti
    5           185        33    zachlatta
    3           165        68    lalitkale
    2           158        38  yehiaabdelm
   29           124        59      bookmtn
   22           123        27     azhenley
   17           105        23  bramadityaw
   18            87         1     defmarco
    4            86        39    holysoles

Total stories scraped: 30
